In [202]:
import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

from rashomon import loss
from rashomon import counter
from rashomon.aggregate import RAggregate_profile, RAggregate
from rashomon.sets import RashomonSet, RashomonProblemCache, RashomonSubproblemCache
from rashomon import tva
from rashomon.extract_pools import extract_pools


%load_ext autoreload
%autoreload 2
# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Multiple profiles

In [170]:
# need a function to map policy indices to profiles

M = 3
R = np.array([3, 3, 3])

num_profiles = 2**M
profiles, profile_map = tva.enumerate_profiles(M)

all_policies = tva.enumerate_policies(M, R)
num_policies = len(all_policies)

policy = (2, 2, 2)
profile = tva.policy_to_profile(policy)
print(profile_map[profile])

7


In [272]:
profiles

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

### Generate data
Setup profiles

In [203]:
# Profile 0: (0, 0, 0)
sigma_0 = None
mu_0 = np.array([0])

# Profile 1: (0, 0, 1)
sigma_1 = np.array([[1]])
mu_1 = np.array([1])

# Profile 2: (0, 1, 0)
sigma_2 = np.array([[1]])
mu_2 = np.array([1])

# Profile 3: (0, 1, 1)
sigma_3 = np.array([[1, 1],
                    [1, 1]])
mu_3 = np.array([4])

# Profile 4: (1, 0, 0)
sigma_4 = np.array([[0]])
mu_4 = np.array([1, 4])

# Profile 5: (1, 0, 1)
sigma_5 = np.array([[1, 1],
                    [1, 1]])
mu_5 = np.array([4])

# Profile 6: (1, 1, 0)
sigma_6 = np.array([[1, 1],
                    [1, 1]])
mu_6 = np.array([4])

# Profile 7: (1, 1, 1)
sigma_7 = np.array([[1, 1],
                    [1, 1],
                    [1, 1]])
mu_7 = np.array([4])

sigma = [sigma_0, sigma_1, sigma_2, sigma_3, sigma_4,
         sigma_5, sigma_6, sigma_7]
mu = [mu_0, mu_1, mu_2, mu_3, mu_4, mu_5, mu_6, mu_7]
var = [1] * num_policies

policies_profiles = {}
pi_policies = {}
for k, profile in enumerate(profiles):

    policies_temp = [(i, x) for i, x in enumerate(all_policies) if tva.policy_to_profile(x) == profile]
    unzipped_temp = list(zip(*policies_temp))
    policies_k = list(unzipped_temp[1])
    policies_profiles[k] = deepcopy(policies_k)

    
    profile_mask = list(map(bool, profile))

    # Mask the empty arms
    for idx, pol in enumerate(policies_k):
        policies_k[idx] = tuple([pol[i] for i in range(M) if profile_mask[i]])

    if np.sum(profile) > 0:
        pi_pools_k, pi_policies_k = extract_pools(policies_k, sigma[k])
        pi_policies[k] = pi_policies_k
    else:
        pi_policies[k] = {0: 0}


In [204]:
policies_profiles

{0: [(0, 0, 0)],
 1: [(0, 0, 1), (0, 0, 2)],
 2: [(0, 1, 0), (0, 2, 0)],
 3: [(0, 1, 1), (0, 1, 2), (0, 2, 1), (0, 2, 2)],
 4: [(1, 0, 0), (2, 0, 0)],
 5: [(1, 0, 1), (1, 0, 2), (2, 0, 1), (2, 0, 2)],
 6: [(1, 1, 0), (1, 2, 0), (2, 1, 0), (2, 2, 0)],
 7: [(1, 1, 1),
  (1, 1, 2),
  (1, 2, 1),
  (1, 2, 2),
  (2, 1, 1),
  (2, 1, 2),
  (2, 2, 1),
  (2, 2, 2)]}

In [269]:
# Random data

np.random.seed(3)

# mu = np.random.uniform(0, 4, size=num_pools)
# mu = np.array([3, 6, 1, 4])
# mu = [0] * num_policies
# var = [1] * num_policies

n_per_pol = 100

num_data = num_policies * n_per_pol
X = np.zeros(shape=(num_data, M))
D = np.zeros(shape=(num_data, 1), dtype='int_')
y = np.zeros(shape=(num_data, 1))

idx_ctr = 0
for k, profile in enumerate(profiles):
    policies_k = policies_profiles[k]

    for idx, policy in enumerate(policies_k):
        profile_id = tva.policy_to_profile(policy)
        pool_id = pi_policies[k][idx]
        # pool_i = pi_policies[idx]
        mu_i = mu[k][pool_id]
        var_i = var[idx]
        y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))
    
        start_idx = idx_ctr * n_per_pol
        end_idx = (idx_ctr + 1) * n_per_pol
    
        X[start_idx:end_idx, ] = policy
        # D[start_idx:end_idx, ] = idx
        D[start_idx:end_idx, ] = idx_ctr
        y[start_idx:end_idx, ] = y_i
    
        idx_ctr += 1
    

In [270]:
R_set, rashomon_profiles = RAggregate(M, R, 9, D, y, 10, reg=1e-2)

R_set

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 2],
 [0, 0, 0, 0, 0, 0, 1, 3],
 [0, 0, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 0, 0, 0, 2, 1],
 [0, 0, 0, 0, 0, 0, 2, 2],
 [0, 0, 0, 0, 0, 0, 2, 3],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 2],
 [0, 0, 0, 0, 0, 1, 0, 3],
 [0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 1, 2],
 [0, 0, 0, 0, 0, 1, 1, 3],
 [0, 0, 0, 0, 0, 1, 2, 0],
 [0, 0, 0, 0, 0, 1, 2, 1],
 [0, 0, 0, 0, 0, 1, 2, 2],
 [0, 0, 0, 0, 0, 1, 2, 3],
 [0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 1],
 [0, 0, 0, 0, 0, 2, 0, 2],
 [0, 0, 0, 0, 0, 2, 0, 3],
 [0, 0, 0, 0, 0, 2, 1, 0],
 [0, 0, 0, 0, 0, 2, 1, 1],
 [0, 0, 0, 0, 0, 2, 1, 2],
 [0, 0, 0, 0, 0, 2, 1, 3],
 [0, 0, 0, 0, 0, 2, 2, 0],
 [0, 0, 0, 0, 0, 2, 2, 1],
 [0, 0, 0, 0, 0, 2, 2, 2],
 [0, 0, 0, 0, 0, 2, 2, 3],
 [0, 0, 0, 1, 0, 0, 0, 0],
 

In [271]:
for k, profile in enumerate(profiles):
    print(profile)
    print(rashomon_profiles[k])

(0, 0, 0)
[None]
(0, 0, 1)
[array([[0.]]), array([[1.]])]
(0, 1, 0)
[array([[0.]])]
(0, 1, 1)
[array([[1.],
       [0.]]), array([[0.],
       [1.]]), array([[1.],
       [1.]])]
(1, 0, 0)
[array([[0.]])]
(1, 0, 1)
[array([[1.],
       [1.]]), array([[0.],
       [1.]]), array([[1.],
       [0.]])]
(1, 1, 0)
[array([[1.],
       [1.]]), array([[0.],
       [1.]]), array([[1.],
       [0.]])]
(1, 1, 1)
[array([[1.],
       [1.],
       [1.]]), array([[1.],
       [0.],
       [1.]]), array([[0.],
       [1.],
       [1.]]), array([[1.],
       [1.],
       [0.]])]


## Single profile

In [34]:
sigma = np.array([[1, 1, 0],
                  [0, 1, 1],
                  [1, 0, 1]], dtype='float64')

M, n = sigma.shape
R = np.array([5, 5, 5])
profile = (1, 1, 1)

num_policies = np.prod(R-1)
profiles, profile_map = tva.enumerate_profiles(M)
all_policies = tva.enumerate_policies(M, R)
policies = [x for x in all_policies if tva.policy_to_profile(x) == profile]
pi_pools, pi_policies = extract_pools(policies, sigma)

for pool_id, pool in pi_pools.items():
    print(pool_id, ":", [policies[i] for i in pool])

0 : [(1, 1, 1), (1, 1, 2), (2, 1, 1), (2, 1, 2), (3, 1, 1), (3, 1, 2)]
1 : [(1, 1, 3), (1, 1, 4), (2, 1, 3), (2, 1, 4), (3, 1, 3), (3, 1, 4)]
2 : [(1, 2, 1), (1, 2, 2), (1, 3, 1), (1, 3, 2), (1, 4, 1), (1, 4, 2), (2, 2, 1), (2, 2, 2), (2, 3, 1), (2, 3, 2), (2, 4, 1), (2, 4, 2), (3, 2, 1), (3, 2, 2), (3, 3, 1), (3, 3, 2), (3, 4, 1), (3, 4, 2)]
3 : [(1, 2, 3), (1, 2, 4), (1, 3, 3), (1, 3, 4), (1, 4, 3), (1, 4, 4), (2, 2, 3), (2, 2, 4), (2, 3, 3), (2, 3, 4), (2, 4, 3), (2, 4, 4), (3, 2, 3), (3, 2, 4), (3, 3, 3), (3, 3, 4), (3, 4, 3), (3, 4, 4)]
4 : [(4, 1, 1), (4, 1, 2)]
5 : [(4, 1, 3), (4, 1, 4)]
6 : [(4, 2, 1), (4, 2, 2), (4, 3, 1), (4, 3, 2), (4, 4, 1), (4, 4, 2)]
7 : [(4, 2, 3), (4, 2, 4), (4, 3, 3), (4, 3, 4), (4, 4, 3), (4, 4, 4)]


### Generate data

In [30]:
np.random.seed(3)

num_pools = len(pi_pools)
# mu = np.random.uniform(0, 4, size=num_pools)
# mu = np.array([3, 6, 1, 4])
mu = np.array([3, 4, 6, 5, 1, 0, 3, 2])
var = [1] * num_pools

n_per_pol = 10

num_data = num_policies * n_per_pol
X = np.ndarray(shape=(num_data, M))
D = np.ndarray(shape=(num_data, 1), dtype='int_')
y = np.ndarray(shape=(num_data, 1))

for idx, policy in enumerate(policies):
    pool_i = pi_policies[idx]
    mu_i = mu[pool_i]
    var_i = var[pool_i]
    y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))

    start_idx = idx * n_per_pol
    end_idx = (idx + 1) * n_per_pol

    X[start_idx:end_idx, ] = policy
    D[start_idx:end_idx, ] = idx
    y[start_idx:end_idx, ] = y_i
    

In [31]:
# This function needs to called only once
policy_means = loss.compute_policy_means(D, y, num_policies)

# This function needs to be called every time the pools change
mu_pools = loss.compute_pool_means(policy_means, pi_pools)

# This function needs to be called every time the pools change
Q = loss.compute_Q(D, y, sigma, policies, policy_means, 0.1)

print(Q)

1.8126958244093103


In [32]:
i = 0
j = 0

B = loss.compute_B(D, y, sigma, i, j, policies, policy_means, 0.1)
print(B)

1.7862060864721792


### RAggregate

In [33]:
P_set = RAggregate(M, R, 10, D, y, 2, reg=0.1)
print(P_set.size)
P_set.seen(sigma)

7


True

In [58]:
pol_means = loss.compute_policy_means(D, y, num_policies)
true_best = pi_pools[np.argmax(mu)]
min_dosage_best_policy = find_min_dosage(true_best, policies)

for s_i in P_set:
    # print(s_i)
    pi_pools_i, pi_policies_i = extract_pools(policies, s_i)
    pool_means_i = loss.compute_pool_means(pol_means, pi_pools_i)
    
    Q = loss.compute_Q(D, y, s_i, policies, pol_means, reg=0.1)
    y_pred = loss.make_predictions(D, pi_policies_i, pool_means_i)
    sqrd_err = mean_squared_error(y, y_pred)
    pol_max = loss.find_best_policies(D, y_pred)
    iou = loss.intersect_over_union(set(true_best), set(pol_max))

    # Min dosage membership
    min_dosage_present = check_membership(min_dosage_best_policy, pol_max)

    # Best policy difference
    best_pol_diff = np.max(mu) - np.max(pool_means_i)
    
    # print(np.max(y_pred), pool_means)
    print(f"Num pools: {len(pi_pools_i)}, Squared loss: {sqrd_err:0.5f}, Q: {Q:0.5f}")
    print(f"Best policy IOU: {iou:.3f}")
    print(f"Min dosage: {min_dosage_present}")
    print(f"Best policy error: {best_pol_diff}")
    print("---")

Num pools: 4, Squared loss: 1.30029, Q: 1.70029
Best policy IOU: 0.500
Min dosage: True
Best policy error: 0.4471226717855128
---
Num pools: 6, Squared loss: 1.29973, Q: 1.89973
Best policy IOU: 0.400
Min dosage: True
Best policy error: 0.4251355538334627
---
Num pools: 6, Squared loss: 1.29852, Q: 1.89852
Best policy IOU: 0.400
Min dosage: False
Best policy error: 0.40918740978712886
---
Num pools: 6, Squared loss: 1.29790, Q: 1.89790
Best policy IOU: 0.250
Min dosage: True
Best policy error: 0.43686580234869066
---
Num pools: 8, Squared loss: 1.01270, Q: 1.81270
Best policy IOU: 1.000
Min dosage: True
Best policy error: -0.09651153607163465
---
Num pools: 6, Squared loss: 1.29657, Q: 1.89657
Best policy IOU: 0.400
Min dosage: True
Best policy error: 0.398387635947028
---
Num pools: 8, Squared loss: 1.18809, Q: 1.98809
Best policy IOU: 0.667
Min dosage: True
Best policy error: 0.2681464021581057
---


### LASSO - Beta -> alpha

In [35]:
G = tva.alpha_matrix(policies)
# print(G)
D_matrix = tva.get_dummy_matrix(D, G, num_policies)

In [64]:
reg_param = 1e-1
mod1 = linear_model.Lasso(reg_param)
mod1.fit(D_matrix, y)
alpha_est = mod1.coef_
y_tva = mod1.predict(D_matrix)
sqrd_err = mean_squared_error(y_tva, y)
print(sqrd_err)
L1_tva = sqrd_err + reg_param * np.linalg.norm(alpha_est, ord=1)
Q_tva = sqrd_err + reg_param * np.linalg.norm(alpha_est, ord=0)
print(Q_tva, L1_tva)

1.300375775335072
1.800375775335072 1.7425677924571503


In [65]:
# print(pi_pools[1])
# np.unique(D[np.where(y_tva == np.max(y_tva)), ])
tva_best = loss.find_best_policies(D, y_tva)
iou_tva = loss.intersect_over_union(set(true_best), set(tva_best))
print(iou_tva)
min_dosage_present_tva = check_membership(min_dosage_best_policy, pol_max)
print(min_dosage_present_tva)
best_policy_error_tva = np.max(mu) - np.max(y_tva)
print(best_policy_error_tva)

1.0
True
0.5628253327386021


In [52]:
np.unique(y_tva)

array([1.80926068, 3.08160831, 4.09466866, 4.25545291, 4.52180272,
       5.36701629, 5.52780054, 5.79415035])

In [53]:
print(pi_pools[3])
np.unique(D[np.where(y_tva == np.unique(y_tva)[2]), ])

[52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]


array([52, 56, 60])

### Varying R

In [192]:
# Idea: np.inf for arbitrary dosage?
sigma = np.array([[1, 0, 1],
                  [0, 1, np.inf]], dtype='float64')
# sigma = np.array([[0, 0, 0],
#                   [0, 0, np.inf]], dtype='float64')

M, n = sigma.shape
R = np.array([5, 4])

num_policies = np.prod(R-1)
policies = enumerate_policies(M, R)
pi_pools, pi_policies = extract_pools(policies, sigma)

for pool_id, pool in pi_pools.items():
    print(pool_id, ":", [policies[i] for i in pool])

0 : [(1, 1), (2, 1)]
1 : [(1, 2), (1, 3), (2, 2), (2, 3)]
2 : [(3, 1), (4, 1)]
3 : [(3, 2), (3, 3), (4, 2), (4, 3)]


In [193]:
counter.num_pools(sigma)

4.0

In [194]:
counter.num_admissible_poolings(4, M, R)

6

In [195]:
np.random.seed(3)

num_pools = len(pi_pools)
mu = np.random.uniform(0, 4, size=num_pools)
var = [1] * num_pools

n_per_pol = 10

num_data = num_policies * n_per_pol
X = np.ndarray(shape=(num_data, M))
D = np.ndarray(shape=(num_data, 1), dtype='int_')
y = np.ndarray(shape=(num_data, 1))

for idx, policy in enumerate(policies):
    pool_i = pi_policies[idx]
    mu_i = mu[pool_i]
    var_i = var[pool_i]
    y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))

    start_idx = idx * n_per_pol
    end_idx = (idx + 1) * n_per_pol

    X[start_idx:end_idx, ] = policy
    D[start_idx:end_idx, ] = idx
    y[start_idx:end_idx, ] = y_i

In [196]:
policy_means = loss.compute_policy_means(D, y, num_policies)
policy_means

array([[22.3306393 , 10.        ],
       [26.09289987, 10.        ],
       [26.9056259 , 10.        ],
       [14.77744099, 10.        ],
       [27.87404021, 10.        ],
       [29.12185777, 10.        ],
       [14.06728243, 10.        ],
       [21.12531734, 10.        ],
       [15.25551251, 10.        ],
       [12.69909817, 10.        ],
       [21.11531527, 10.        ],
       [19.32885624, 10.        ]])

In [197]:
Q = loss.compute_Q(D, y, sigma, policies, policy_means, reg=1)
Q

4.9889861406550375

In [198]:
i = 1
j = 1
B = loss.compute_B(D, y, sigma, i, j, policies, policy_means, reg=1)
print(B)

6.975889428946971


In [199]:
P_set = RAggregate(2, R, 4, D, y, 1.4, reg=0.1)
print(P_set.size)
P_set.seen(sigma)

5


True